In [ ]:
!unzip 'drive/My Drive/TIES4911/task4/apparel-images-dataset.zip' -d '/content/drive/My Drive/TIES4911/task4/apparel-images-dataset'

In [27]:
import tensorflow as tf
import pathlib
from tensorflow.keras.applications import mobilenet_v2
from keras import Input, Model, layers
from datetime import datetime
import os

In [28]:
data_dir = pathlib.Path('drive/My Drive/TIES4911/task4/apparel-images-dataset')
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

11385


In [29]:
batch_size = 32
img_height = 224
img_width = 224

In [ ]:
(train_ds, val_ds) = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    shuffle=True,
    validation_split=0.2,
    subset='both',
    image_size=(img_height, img_width),
    batch_size=batch_size,
    seed=123,
    label_mode='categorical'
)

Found 11385 files belonging to 24 classes.
Using 9108 files for training.
Using 2277 files for validation.


In [ ]:
class_names = train_ds.class_names
num_classes = len(class_names)
print(num_classes)

24


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [36]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

In [37]:
inputs = Input(shape=(img_height, img_width, 3))

base_model = mobilenet_v2.MobileNetV2(input_tensor=inputs)
base_model.trainable = False
preprocess_input = mobilenet_v2.preprocess_input
output_layer = layers.Dense(num_classes, activation='softmax')

x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
outputs = output_layer(x)

model = Model(inputs, outputs)
model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(),
        metrics=["accuracy"])
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.math.truediv_2 (TFOpLamb  (None, 224, 224, 3)      0         
 da)                                                             
                                                                 
 tf.math.subtract_2 (TFOpLam  (None, 224, 224, 3)      0         
 bda)                                                            
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 1000)             3538984   
 ional)                                                          
                                                           

In [38]:
checkpoint_path = 'drive/My Drive/TIES4911/task4/task4-3/checkpoint-{epoch:04d}'
checkpoint_dir = os.path.dirname(checkpoint_path)
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

drive/My Drive/TIES4911/task4/task4-3/checkpoint-0020


In [39]:
model.load_weights(latest)
model.evaluate(val_ds, verbose=2)

72/72 - 105s - loss: 1.6342 - accuracy: 0.4717 - 105s/epoch - 1s/step


[1.6342331171035767, 0.47167325019836426]

In [41]:
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'drive/My Drive/TIES4911/task4/task4-3/logs/%s' % stamp
tensorboard = tf.keras.callbacks.TensorBoard(logdir)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    save_freq=image_count // batch_size * 2,
    verbose=1)

history = model.fit(train_ds,
                    epochs=30,
                    verbose=1,
                    initial_epoch=20,
                    validation_data=val_ds,
                    callbacks=[tensorboard, model_checkpoint_callback])

Epoch 21/30
285/285 [==============================] - 586s 2s/step - loss: 1.9924 - accuracy: 0.4111 - val_loss: 1.6207 - val_accuracy: 0.4752
Epoch 22/30
285/285 [==============================] - 581s 2s/step - loss: 1.9404 - accuracy: 0.4357 - val_loss: 1.6109 - val_accuracy: 0.4734
Epoch 23/30
139/285 [=============>................] - ETA: 3:45 - loss: 1.9068 - accuracy: 0.4432
Epoch 23: saving model to drive/My Drive/TIES4911/task4/task4-3/checkpoint-0023
285/285 [==============================] - 582s 2s/step - loss: 1.9081 - accuracy: 0.4429 - val_loss: 1.6020 - val_accuracy: 0.4717
Epoch 24/30
285/285 [==============================] - 537s 2s/step - loss: 1.8848 - accuracy: 0.4441 - val_loss: 1.5931 - val_accuracy: 0.4734
Epoch 25/30
279/285 [============================>.] - ETA: 9s - loss: 1.8656 - accuracy: 0.4574 
Epoch 25: saving model to drive/My Drive/TIES4911/task4/task4-3/checkpoint-0025
285/285 [==============================] - 586s 2s/step - loss: 1.8668 - accura

In [ ]:
model.evaluate(val_ds, verbose=2)

72/72 - 108s - loss: 1.6336 - accuracy: 0.4730 - 108s/epoch - 1s/step


[1.6336146593093872, 0.4729907810688019]

In [ ]:
model.save("drive/My Drive/TIES4911/task4/task4-3/model")